# Weather API 


In this NB we import the weather data, preprocess if is needed and save as a new Data Frame. 


### Historical Data 

For training and testing the model we use the historicla weather data related to our energy consumption data provided. 

For that, we'll use the API provided by [AT-Wetter](http://at-wetter.tk/index.php?men=api). We'll use this one, and not the OpenWeather API since if we want to get the historical data depth more than a week (we need a year) we'll have to use several queries. 


#### API 

This API don't need a Key or other autentication. It's located on the AT-Server in `http://at-wetter.tk/api/v1/.`
The instruction to use it are in their [website](http://at-wetter.tk/index.php?men=api) 

#### Example 

To query **all the available stations** we need a call following the format: 
`/api/v1/[field]/[YYYY-MM-DD]/[day count]` 

A query call to `http://at-wetter.tk/api/v1/t/2014-08-01/2`  returns Temperature Data for every available station for 2014-08-01, 2 days in the past. 



To query just **one specific stations** we use: 

`/api/v1/station/[station]/[field]|[all]/[YYYY-MM-DD]/[day count]`

This will return the `[field]` from the specified date `[YYYY-MM-DD]` and prints data `[day count]` days in the past for the specified stations `[station]`



### Calling the API


In [13]:
import requests as rq
import json
import datetime as dt
import pandas as pd

* See the available stations 

In [ ]:
URL ="http://at-wetter.tk/api/v1/stations"
response = rq.request("GET", URL)
print(response.text)

* Calculate de day count from Building Energy Data set 

We cold do it manually, but in order to automated all the steps we'll calculated with a python script 

In [ ]:
import pandas as pd 
DataFrame = pd.read_excel("./DataSets/Building_energy.xlsx", header=1)
DataFrame.head()

#Calculate the Delta Time 

start_date = dt.datetime.strptime(DataFrame['Time'][0], "%Y-%m-%d %H:%M:%S")
end_date = dt.datetime.strptime(DataFrame['Time'][len(DataFrame)-1], "%Y-%m-%d %H:%M:%S")
day_count = end_date - start_date
print(day_count.days)


As the `[daycount]` field for API Query doesn't allow hours, if we have any hours of difference between the start day and the end day, we add an extra day. Then in the data frame matching we'll delethe the extra hours for the extra day. 

In [ ]:
dayCount=0 
if day_count.seconds //3600 > 0: 
    dayCount = day_count.days + 1
else: 
    dayCount = day_count.days
print(dayCount)

* Download the temperature data

In [ ]:
station = '11240' #GRAZ 
field = 'all'  #Temperatuer
days_count = str(day_count.days)

URL = "http://at-wetter.tk/api/v1/station/"
URL += station 
URL += "/" + field
URL += "/" + str(end_date.year) + "-" + str(end_date.month) + "-" + str(end_date.day)
URL += "/" + str(dayCount)
response = rq.request("GET", URL)

print(URL)
#print(response.text)
type(response.text)


In [ ]:
#response.json(encoding="utf-8")
#print(response.text)

* Saving the data into a Data Frame 

Since the data can't be downloaded in JSON format, we first split the whole text into lineas, and then elements. 

In [ ]:
lines = response.text.splitlines()
ColumnNames = [ (col.replace("'",'')) for col in lines[0].split(";") ]
WeatDataFrame = pd.DataFrame(columns=ColumnNames, index=range(len(lines)-2)) # -1 cause the columnames and -1 again cuase it starts from 0 

The index of the daframe starts from 0 but the first line of response.text is the ColumnName row. So we start from 0 for the DataFrame index and for idx+1 for lineas

In [ ]:
for idx in range(len(lines)-2):  
    WeatDataFrame.iloc[idx] = [ (col.replace("'",'')) for col in lines[idx+1].split(";") ]
WeatDataFrame

* Changing the datum and zeit column into a newone with datetime type object

In [ ]:
WeatDataFrame.info()

In [ ]:
WeatDataFrame["datum"][:]

In [ ]:
from datetime import datetime as dt 

dates=[] 
for idx in range(len(WeatDataFrame["datum"])): 
    str_date = str(WeatDataFrame["datum"][idx]) + " " + str(WeatDataFrame["zeit"][idx])
    dt = dt.strptime(str_date, "%Y-%m-%d %H:%M")
    dates.append(dt)

print(len(dates), len(WeatDataFrame.index))
WeatDataFrame.insert(0,"Time",dates)

In [ ]:
WeatDataFrame

* Deleting the timestamps column, zatum and zeit 

In [ ]:
WeatDataFrame.drop('datum', inplace = True, axis=1) 
WeatDataFrame.drop('zeit', inplace = True, axis=1) 
WeatDataFrame.drop('timestamp', inplace = True, axis=1) 


Save the data frame into a csv file 

In [5]:
WeatDataFrame.to_csv("./DataSets/Raw_Weather_DataSet.csv",index=False)

NameError: name 'WeatDataFrame' is not defined

### Forecast 
#### API 

We'll use python to request de dataset from the OpenWeather data base. The API  request call should be in this format: 

``` http://history.openweathermap.org/data/2.5/history/city?q={city ID},{country code}&type=hour&start={start}&end={end}&appid={API key}```

#### Usage 

Once we introduce the day that we want to predict we request for that day's forecast. If we have it, we arrange all the necessarry inputs for that day (temp, holiday, week day,etc) and we introduce them into the model. 

In [2]:
API_Key = "8b99cd46b801c25195aef4cca39daee5"

In [18]:
URL = " http://api.openweathermap.org/data/2.5/forecast?q=Graz&appid=" + API_Key
response = rq.request("GET", URL)
print(response.text)

{"cod":"200","message":0,"cnt":40,"list":[{"dt":1637539200,"main":{"temp":276.03,"feels_like":276.03,"temp_min":276.03,"temp_max":277.57,"pressure":1013,"sea_level":1013,"grnd_level":969,"humidity":73,"temp_kf":-1.54},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":90},"wind":{"speed":0.75,"deg":355,"gust":0.75},"visibility":10000,"pop":0,"sys":{"pod":"n"},"dt_txt":"2021-11-22 00:00:00"},{"dt":1637550000,"main":{"temp":276.4,"feels_like":276.4,"temp_min":276.4,"temp_max":277.15,"pressure":1013,"sea_level":1013,"grnd_level":970,"humidity":78,"temp_kf":-0.75},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04n"}],"clouds":{"all":93},"wind":{"speed":0.49,"deg":338,"gust":0.48},"visibility":10000,"pop":0,"sys":{"pod":"n"},"dt_txt":"2021-11-22 03:00:00"},{"dt":1637560800,"main":{"temp":276.63,"feels_like":276.63,"temp_min":276.63,"temp_max":276.93,"pressure":1014,"sea_level":1014,"grnd_level":971,"humidity":83,

In [31]:
import jmespath
expression = jmespath.compile('list[].weather')
res = expression.search(response.json())

pd.DataFrame(res)

,0
0,"{'id': 804, 'main': 'Clouds', 'description': '..."
1,"{'id': 804, 'main': 'Clouds', 'description': '..."
2,"{'id': 804, 'main': 'Clouds', 'description': '..."
3,"{'id': 804, 'main': 'Clouds', 'description': '..."
4,"{'id': 804, 'main': 'Clouds', 'description': '..."
5,"{'id': 804, 'main': 'Clouds', 'description': '..."
6,"{'id': 804, 'main': 'Clouds', 'description': '..."
7,"{'id': 804, 'main': 'Clouds', 'description': '..."
8,"{'id': 804, 'main': 'Clouds', 'description': '..."
9,"{'id': 803, 'main': 'Clouds', 'description': '..."
